In [1]:
#!pip install eventregistry
from eventregistry import *
import pandas as pd
import json

In [22]:
# Load the API key from the JSON file
with open("config.json", "r") as file:
    config = json.load(file)
api_key = config["api_key"]

# Initialize EventRegistry with the API key
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)

# Define topics to search for
topics = [
    "Donald Trump", "Kamala Harris", "Israel", "Palestine", "Palestinians", "Hamas", "FEMA", "Abortion",
    "Inflation", "Unemployment", "Economy", "Dockworkers", "ILA Port", "Immigration"
]

# Define sources to search for and get their URIs
source_names = ["NPR", "MSNBC", "AP News", "FOX", "Forbes"]
source_uris = {source: er.getNewsSourceUri(source) for source in source_names}

# List to store the names of all generated DataFrames
dataframe_names = []

# Loop through each topic
for topic in topics:
    # Get the URI for the concept
    concept_uri = er.getConceptUri(topic)
    
    # List to hold all articles' data for the current topic (across all sources)
    articles_data = []
    
    # Loop through each source individually
    for source_name, source_uri in source_uris.items():
        # Define the query for each topic and source
        q = QueryArticlesIter(
            conceptUri=concept_uri,
            sourceUri=source_uri,
            sourceLocationUri=er.getLocationUri("United States"),  # Only US sources
        )

        # Fetch and accumulate up to 500 articles for the current topic from this source
        for art in q.execQuery(er, sortBy="date", maxItems=500):
            articles_data.append({
                "title": art.get("title"),
                "source": art.get("source", {}).get("title"),
                "author": art.get("author"),
                "url": art.get("url"),
                "publishedAt": art.get("dateTime"),
                "content": art.get("body")
            })

    # Create a single DataFrame for the current topic with articles from all sources
    articles_df = pd.DataFrame(articles_data)
    
    # Save the DataFrame to a CSV file
    file_name = f"{topic.replace(' ', '_')}_articles.csv"
    articles_df.to_csv(file_name, index=False)

    # Dynamically set the DataFrame name based on the topic, replacing spaces with underscores
    df_name = f"{topic.replace(' ', '_')}_df"
    globals()[df_name] = articles_df

    # Append the DataFrame name to the list
    dataframe_names.append(df_name)

# Print the list of all generated DataFrame names
print("Generated DataFrames:", dataframe_names)

Generated DataFrames: ['Donald_Trump_df', 'Kamala_Harris_df', 'Israel_df', 'Palestine_df', 'Palestinians_df', 'Hamas_df', 'FEMA_df', 'Abortion_df', 'Inflation_df', 'Unemployment_df', 'Economy_df', 'Dockworkers_df', 'ILA_Port_df', 'Immigration_df']


In [23]:
# Example: Access the "Donald Trump" DataFrame
Donald_Trump_df.head() # type: ignore

,title,source,author,url,publishedAt,content
0,The politics of natural disasters : The NPR Po...,NPR,None,https://www.npr.org/2024/10/10/1210938546/poli...,2024-10-10T21:01:48Z,The politics of natural disasters : The NPR Po...
1,"How The Internet, Social Media, And Podcasts A...",NPR,None,https://www.npr.org/2024/10/10/1210938545/1a-1...,2024-10-10T21:01:39Z,Jemele Hill (L) and Senator Kamala Harris spea...
2,"Ethel Kennedy, social activist and widow of Ro...",NPR,None,https://www.npr.org/2024/10/10/nx-s1-5148861/e...,2024-10-10T15:53:45Z,"BOSTON, Mass. -- Ethel Kennedy, the widow of S..."
3,What matters to key swing state voters in Wisc...,NPR,None,https://www.npr.org/2024/10/10/nx-s1-5143006/w...,2024-10-10T13:41:06Z,The Milwaukee Running Group -- OMG meets up on...
4,Politicians say health plans should cover IVF....,NPR,None,https://www.npr.org/2024/10/10/nx-s1-5147815/i...,2024-10-10T13:11:09Z,One round of in vitro fertilization or IVF can...


In [24]:
df = pd.read_csv('Donald_Trump_articles.csv')
df['source'].value_counts()

source
AP NEWS      500
Forbes       500
Fox News     500
MSNBC.com    397
NPR          243
Name: count, dtype: int64

In [25]:
df1 = pd.read_csv('Abortion_articles.csv')
df1['source'].value_counts()

source
Fox News     203
AP NEWS      157
Forbes        70
MSNBC.com     60
NPR           43
Name: count, dtype: int64

In [ ]:
# 1. Merge relevant dataframes - find relevant actors for each --> Mason
# 2. Data Cleaning/LDA --> Carson
# 2. Active/passive/topic voice code --> Timmy 
# 3. ChatGPT sentiment for actors in a given category (-100 to 100) --> Ethan/Timmy
# 4. Coming up with some sort of logical evaluation score metric based on active/passive voice, sentiment, title, and number of articles about certain subjects etc. --> Bethel/Medha (Come up with a few options)